In [16]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [17]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lebu,CL,-37.6167,-73.6500,50.22,85,96,7.40,overcast clouds
1,1,Vao,NC,-22.6667,167.4833,76.24,84,97,11.79,light rain
2,2,Kaitangata,NZ,-46.2817,169.8464,46.11,64,12,2.64,few clouds
3,3,Vanavara,RU,60.3400,102.2797,73.96,59,80,8.25,light rain
4,4,Toora-Khem,RU,52.4675,96.1100,66.51,37,100,4.68,overcast clouds


In [18]:
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))


In [19]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Get the maximum temperature.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 88


In [20]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Vao,NC,-22.6667,167.4833,76.24,84,97,11.79,light rain
3,3,Vanavara,RU,60.3400,102.2797,73.96,59,80,8.25,light rain
7,7,Kapaa,US,22.0752,-159.3190,82.38,81,100,17.27,light rain
9,9,Atuona,PF,-9.8000,-139.0333,78.08,69,0,19.35,clear sky
10,10,Kavieng,PG,-2.5744,150.7967,84.65,69,27,14.47,scattered clouds
12,12,Padang,ID,-0.9492,100.3543,82.45,72,94,3.67,overcast clouds
13,13,Praia,CV,14.9215,-23.5087,75.74,73,75,11.50,broken clouds
17,17,Komsomolskiy,UZ,40.4272,71.7189,72.32,53,99,2.30,overcast clouds
20,20,Avarua,CK,-21.2078,-159.7750,78.85,69,20,6.91,few clouds
23,23,Rikitea,PF,-23.1203,-134.9692,72.59,64,60,2.46,broken clouds


In [21]:
# 4a. Determine if there are any empty rows.
if preferred_cities_df.isnull().values.any() == True:
    print("Null values present in the Preferred cities list")


In [22]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
if preferred_cities_df.isnull().values.any() == True:
    preferred_cities_df = preferred_cities_df.dropna()
preferred_clean_hotel_df = preferred_cities_df


In [23]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_clean_hotel_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Vao,NC,76.24,light rain,-22.6667,167.4833,
3,Vanavara,RU,73.96,light rain,60.3400,102.2797,
7,Kapaa,US,82.38,light rain,22.0752,-159.3190,
9,Atuona,PF,78.08,clear sky,-9.8000,-139.0333,
10,Kavieng,PG,84.65,scattered clouds,-2.5744,150.7967,
12,Padang,ID,82.45,overcast clouds,-0.9492,100.3543,
13,Praia,CV,75.74,broken clouds,14.9215,-23.5087,
17,Komsomolskiy,UZ,72.32,overcast clouds,40.4272,71.7189,
20,Avarua,CK,78.85,few clouds,-21.2078,-159.7750,
23,Rikitea,PF,72.59,broken clouds,-23.1203,-134.9692,


In [24]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame.
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(f"Hotel not found... skipping.{index}")
        

Hotel not found... skipping.3
Hotel not found... skipping.72
Hotel not found... skipping.108
Hotel not found... skipping.121
Hotel not found... skipping.134
Hotel not found... skipping.225
Hotel not found... skipping.285
Hotel not found... skipping.293
Hotel not found... skipping.317
Hotel not found... skipping.321
Hotel not found... skipping.333
Hotel not found... skipping.367
Hotel not found... skipping.400
Hotel not found... skipping.409
Hotel not found... skipping.425
Hotel not found... skipping.430
Hotel not found... skipping.439
Hotel not found... skipping.442
Hotel not found... skipping.455
Hotel not found... skipping.470
Hotel not found... skipping.481
Hotel not found... skipping.491
Hotel not found... skipping.536
Hotel not found... skipping.586
Hotel not found... skipping.610
Hotel not found... skipping.627
Hotel not found... skipping.663
Hotel not found... skipping.674
Hotel not found... skipping.680
Hotel not found... skipping.681
Hotel not found... skipping.687
Hotel not f

In [27]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != "" ]
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Vao,NC,76.24,light rain,-22.6667,167.4833,Hotel Kou-Bugny
7,Kapaa,US,82.38,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
9,Atuona,PF,78.08,clear sky,-9.8000,-139.0333,Villa Enata
10,Kavieng,PG,84.65,scattered clouds,-2.5744,150.7967,Nusa Island Retreat
12,Padang,ID,82.45,overcast clouds,-0.9492,100.3543,Mercure Padang
...,...,...,...,...,...,...,...
690,Kathu,TH,85.69,overcast clouds,7.9178,98.3332,Novotel Phuket Resort
693,Juneau,US,77.00,clear sky,58.3019,-134.4197,"Baranof Downtown, BW Signature Collection"
694,Puerto Narino,CO,71.82,overcast clouds,-3.7703,-70.3831,Eware Refugio Amazonico
695,Tutoia,BR,77.04,overcast clouds,-2.7619,-42.2744,Pousada Guará's


In [28]:
# 8a. Create the output File (CSV)

output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [29]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [30]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))